In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2
import os
import random
import time
from commonfunctions import *
# Depending on library versions on your system, one of the following imports 
from sklearn.model_selection import train_test_split

In [2]:
def eq_hist(img):
    # Step 1: Compute the histogram
    img= img.astype(int)
    h, _ = np.histogram(img, bins=256, range=(0, 256))
    
    # Step 2: Compute the cumulative distribution function (CDF)
    h_c = np.cumsum(h)
    
    # Step 3: Normalize the CDF to the range [0, 255]
    t = np.round(255 * h_c / h_c[-1]).astype(np.uint8)
    
    # Step 4: Map the original pixel values to equalized values
    equalized_img = t[img]
    
    return equalized_img



def gamma_correction(c,gamma,img):
    gamma_img = c*np.power(img,gamma)
    return gamma_img

In [3]:

def detect_hand(image):
    copy = image.copy()
    hsv = cv2.cvtColor(copy, cv2.COLOR_BGR2HSV)

    # Skin color range in HSV
    lower_skin = (0, 20, 70)
    upper_skin = (20, 255, 255)

    # Create mask
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Load face detector
    
    # Morphological operations to clean noise
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Filter by area and aspect ratio
        min_area = 5000
        filtered_contours = [c for c in contours if cv2.contourArea(c) > min_area]

        if filtered_contours:
            hand_contour = max(filtered_contours, key=cv2.contourArea)

            # Convex Hull and Convexity Defects
            hull = cv2.convexHull(hand_contour, returnPoints=False)
            defects = cv2.convexityDefects(hand_contour, hull)

            # Draw the hand contour and convex hull
            cv2.drawContours(copy, [hand_contour], -1, (0, 255, 0), 2)
            cv2.drawContours(copy, [cv2.convexHull(hand_contour)], -1, (255, 0, 0), 2)

            # Highlight defects
            if defects is not None:
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i][0]
                    far = tuple(hand_contour[f][0])
                    cv2.circle(copy, far, 5, (0, 0, 255), -1)

    return copy


In [4]:
import cv2
import numpy as np

def detect_hand(image):
    copy = image.copy()
    hsv = cv2.cvtColor(copy, cv2.COLOR_BGR2HSV)

    # Skin color range in HSV
    lower_skin = (0, 20, 70)
    upper_skin = (20, 255, 255)

    # Create mask for skin color
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Morphological operations to clean noise
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Load the face detector (Haar Cascade Classifier)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in the image
    gray = cv2.cvtColor(copy, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Initialize an empty image for the result
    result = np.zeros_like(image)

    # Find contours for hand detection
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Filter by area and aspect ratio
        min_area = 5000
        filtered_contours = [c for c in contours if cv2.contourArea(c) > min_area]

        if filtered_contours:
            # Loop over the filtered contours and draw bounding boxes
            for contour in filtered_contours:
                # Get bounding rectangle for each contour
                x, y, w, h = cv2.boundingRect(contour)
                # Create a mask for the current rectangle
                hand_mask = np.zeros_like(image)
                cv2.drawContours(hand_mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)

                # Apply the hand mask to the original image
                temp_result = cv2.bitwise_and(image, hand_mask)

                # Copy the result to ensure the hand areas are preserved
                result = cv2.add(result, temp_result)

    # Now mask out the face regions by adding black rectangles in the result
    for (x, y, w, h) in faces:
        cv2.rectangle(result, (x, y), (x + w, y + h), (0, 0, 0), -1)

    return result


In [5]:


def highlight_skin(image):
   # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define skin color range in HSV
    lower_skin = (0, 20, 70)
    upper_skin = (20, 255, 255)

    # Create a mask for skin regions
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a copy of the original image to color the selected regions
    result = image.copy()

    # Iterate over each contour to check aspect ratio and filter by the ratio
    for contour in contours:
        if cv2.contourArea(contour) > 100:  # Filter small contours (optional)
            # Get bounding box of the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Calculate aspect ratio (length / width)
            aspect_ratio = float(w) / float(h)  # Length (height) / Width

            # Check if the aspect ratio matches the desired threshold
            if 0.5 <= aspect_ratio <= 0.8:  # Adjust the range if necessary
                # If it matches, color the region green
                cv2.drawContours(result, [contour], -1, (0, 255, 0), -1)  # Fill the contour with green

    return result


def detect_hand_using_fingers(image):
    copy = image.copy()
    hsv = cv2.cvtColor(copy, cv2.COLOR_BGR2HSV)

    # Skin color range in HSV
    lower_skin = (0, 20, 70)
    upper_skin = (20, 255, 255)

    # Create mask for skin color
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Morphological operations to clean noise
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours for hand detection
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Filter by area and aspect ratio
        min_area = 5000
        filtered_contours = [c for c in contours if cv2.contourArea(c) > min_area]

        if filtered_contours:
            # Get the largest contour (assumed to be the hand)
            hand_contour = max(filtered_contours, key=cv2.contourArea)

            # Approximate the contour to make it simpler (remove small jaggedness)
            epsilon = 0.02 * cv2.arcLength(hand_contour, True)
            approx_contour = cv2.approxPolyDP(hand_contour, epsilon, True)

            # Get convex hull and convexity defects to detect fingers
            hull = cv2.convexHull(approx_contour, returnPoints=False)

            # Ensure the convex hull is valid
            if len(hull) > 3:  # At least 4 points needed for convexity defects
                defects = cv2.convexityDefects(hand_contour, hull)

                # Draw convex hull
                cv2.drawContours(copy, [cv2.convexHull(hand_contour)], -1, (0, 255, 0), 2)

                if defects is not None:
                    # Loop through defects to find finger tips
                    finger_tips = []
                    for i in range(defects.shape[0]):
                        s, e, f, d = defects[i][0]
                        far = tuple(hand_contour[f][0])
                        finger_tips.append(far)

                        # Draw the finger tips
                        cv2.circle(copy, far, 5, (0, 0, 255), -1)

                    # If there are finger tips, use them to define the hand region
                    if finger_tips:
                        # Use finger tips to estimate the upper region of the hand
                        # Find the top-most point among finger tips
                        finger_tips.sort(key=lambda x: x[1])  # Sort by y-coordinate (topmost first)
                        top_finger = finger_tips[0]  # This is the top-most finger tip

                        # Assuming the hand's region is above this finger
                        x, y, w, h = cv2.boundingRect(hand_contour)
                        extended_y = max(y - 50, 0)  # Extend upwards from the palm
                        cv2.rectangle(copy, (x, extended_y), (x + w, y + h), (0, 255, 255), 2)

    return copy


In [6]:

def highlight_thin_skin(image):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define skin color range in HSV
    lower_skin = (0, 20, 70)
    upper_skin = (20, 255, 255)

    # Create a mask for skin regions
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a copy of the original image to color the selected regions
    result = image.copy()

    # Iterate over each contour to check aspect ratio dynamically
    for contour in contours:
        if cv2.contourArea(contour)>1000 and cv2.contourArea(contour)<10000:  # Filter small contours
            # Fit a minimum area rotated rectangle around the contour
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)  # Get the 4 points of the rectangle
            box = np.int0(box)  # Convert to integer
            #print(contour)
            # Calculate the width and height of the rectangle
            width = rect[1][0]
            height = rect[1][1]

            # Ensure width is always smaller than height

            # Calculate the aspect ratio (length / width)
            aspect_ratio =  width/height if width > 0 else 0
            
            print(aspect_ratio)
            # Check if the aspect ratio matches the desired threshold
            if 0.5 <= aspect_ratio <= 0.9:  # Adjust the range if necessary
                # Draw the contour or rectangle on the result image
                cv2.drawContours(result, [box], -1, (0, 255, 0), 2)  # Green rectangle
                cv2.drawContours(result, [contour], -1, (0, 255, 0), -1)  # Fill the contour with green

    return result


In [35]:
import cv2
import numpy as np

def detect_hand_with_enhancements(image):
    # Step 1: Preprocessing (Blur to reduce noise)
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    # Step 2: Dynamic Skin-Tone Detection
    lower_skin = np.array([0, 30, 60], dtype=np.uint8)
    upper_skin = np.array([20, 150, 255], dtype=np.uint8)
    skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Step 3: Adaptive Edge Detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 100)

    # Ensure edges are connected
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # Step 4: Contour Detection on Skin Mask
    contours, _ = cv2.findContours(skin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filled_mask = np.zeros_like(edges)

    # Create a mask with additional padding for floodFill
    flood_fill_mask = np.zeros((edges.shape[0] + 2, edges.shape[1] + 2), dtype=np.uint8)
    flood_fill_mask[1:-1, 1:-1] = edges
    # Overlay for displaying centroids
    centroid_overlay = image.copy()

    # Skin tone mean for comparison
    skin_tone_mean = np.array([10, 100, 150])  # Mean skin tone in HSV
    threshold = 25  # Acceptable difference threshold

    for contour in contours:
        # Filter small contours
        if cv2.contourArea(contour) > 3000:
            # Calculate the centroid of the contour
            M = cv2.moments(contour)
            if M["m00"] != 0:  # Avoid division by zero
                x = int(M["m10"] / M["m00"])
                y = int(M["m01"] / M["m00"])

                # Draw the centroid point on the overlay
                cv2.circle(centroid_overlay, (x, y), 5, (0, 0, 255), -1)  # Red circle for centroid

                # Calculate the average color inside the contour
                mask = np.zeros(image.shape[:2], dtype=np.uint8)
                cv2.drawContours(mask, [contour], -1, 255, -1)  # Create a mask of the contour
                mean_color = cv2.mean(hsv, mask=mask)[:3]

                # Check if the mean color is within the threshold
                if np.all(np.abs(np.array(mean_color) - skin_tone_mean) < threshold):
                    # Flood-fill the region using the modified flood-fill mask
                    cv2.floodFill(filled_mask, flood_fill_mask, seedPoint=(x, y), newVal=255)

    # Step 5: Morphological Operations (Post-Processing)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    filled_mask = cv2.morphologyEx(filled_mask, cv2.MORPH_CLOSE, kernel)  # Close small gaps
    filled_mask = cv2.morphologyEx(filled_mask, cv2.MORPH_OPEN, kernel)   # Remove noise

    # Combine the filled regions with the original image
    result = cv2.bitwise_and(image, image, mask=filled_mask)

    return result, filled_mask, edges, centroid_overlay

# Example usage
def main():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process frame
        result, filled_mask, edges, centroid_overlay = detect_hand_with_enhancements(frame)
        
        # Display results
        cv2.imshow("Original", frame)
        cv2.imshow("Region-Filled Hand", result)
        cv2.imshow("Edges (Connected)", edges)
        cv2.imshow("Filled Mask", filled_mask)
        cv2.imshow("Centroids Overlay", centroid_overlay)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
